In [19]:
import requests
import pandas as pd
from datetime import datetime
import pytz
from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, String, Date, Time

def convert_to_est(time_string):
    if time_string is None:
        return None  # Return None if the time_string is None
    
    # Add a placeholder date to make the time string a valid ISO 8601 format
    # Assuming the time is UTC (with '+00:00' offset)
    time_string = "1970-01-01T" + time_string
    
    try:
        # Parse the time string into a datetime object
        utc_time = datetime.fromisoformat(time_string)
        
        # Convert UTC to EST (Eastern Standard Time)
        est_time = utc_time.astimezone(pytz.timezone('US/Eastern'))
        
        # Return only the time part as a string (remove the date)
        return est_time.strftime('%H:%M:%S')
    
    except ValueError as e:
        print(f"Error in convert_to_est: {e}")
        return None

# Function to extract session date
def extract_session_date(session_data):
    if isinstance(session_data, dict) and 'date' in session_data:
        return session_data.get('date', None)
    return None

url2 = 'https://api.jolpi.ca/ergast/f1/2024/races/'
response = requests.get(url2)

data = response.json()  
# Extract races list
races = data.get('MRData', {}).get('RaceTable', {}).get('Races', [])
# Convert to DataFrame
df = pd.DataFrame(races)

schedule = []
for race in races:
    round_number = race.get('round', None)  
    race_name = race.get('raceName', None)  
    circuit = race.get('Circuit', {}).get('circuitName', None)  
    location = race.get('Circuit', {}).get('Location', {})
    locality = location.get('locality', None)  
    country = location.get('country', None)  
    race_date = race.get('date', None)  
    race_time = convert_to_est(race.get('time', None))  

    # Extract and convert session times and dates
    first_practice_date = extract_session_date(race.get('FirstPractice', {}))
    first_practice_time = convert_to_est(race.get('FirstPractice', {}).get('time', None))  
    
    second_practice_date = extract_session_date(race.get('SecondPractice', {}))
    second_practice_time = convert_to_est(race.get('SecondPractice', {}).get('time', None))  
    
    third_practice_date = extract_session_date(race.get('ThirdPractice', {}))
    third_practice_time = convert_to_est(race.get('ThirdPractice', {}).get('time', None))  
    
    qualifying_date = extract_session_date(race.get('Qualifying', {}))
    qualifying_time = convert_to_est(race.get('Qualifying', {}).get('time', None))  
    
    sprint_date = extract_session_date(race.get('Sprint', {}))
    sprint_time = convert_to_est(race.get('Sprint', {}).get('time', None))  

    # Append extracted data with Date first and then Time
    schedule.append([round_number, race_name, circuit, f"{locality}, {country}", race_date, race_time,
                     first_practice_date, first_practice_time, second_practice_date, second_practice_time,
                     third_practice_date, third_practice_time, qualifying_date, qualifying_time,
                     sprint_date, sprint_time])

In [20]:
# Convert to DataFrame
df = pd.DataFrame(schedule, columns=['round', 'race', 'circuit', 'location', 'race_date', 'race_time',
                                             'first_practice_date', 'first_practice_time', 
                                             'second_practice_date', 'second_practice_time',
                                             'third_practice_date', 'third_practice_time', 
                                             'qualifying_date', 'qualifying_time', 
                                             'sprint_date', 'sprint_time'])
df

,round,race,circuit,location,race_date,race_time,first_practice_date,first_practice_time,second_practice_date,second_practice_time,third_practice_date,third_practice_time,qualifying_date,qualifying_time,sprint_date,sprint_time
0,1,Bahrain Grand Prix,Bahrain International Circuit,"Sakhir, Bahrain",2024-03-02,10:00:00,2024-02-29,06:30:00,2024-02-29,10:00:00,2024-03-01,07:30:00,2024-03-01,11:00:00,None,None
1,2,Saudi Arabian Grand Prix,Jeddah Corniche Circuit,"Jeddah, Saudi Arabia",2024-03-09,12:00:00,2024-03-07,08:30:00,2024-03-07,12:00:00,2024-03-08,08:30:00,2024-03-08,12:00:00,None,None
2,3,Australian Grand Prix,Albert Park Grand Prix Circuit,"Melbourne, Australia",2024-03-24,23:00:00,2024-03-22,20:30:00,2024-03-22,00:00:00,2024-03-23,20:30:00,2024-03-23,00:00:00,None,None
3,4,Japanese Grand Prix,Suzuka Circuit,"Suzuka, Japan",2024-04-07,00:00:00,2024-04-05,21:30:00,2024-04-05,01:00:00,2024-04-06,21:30:00,2024-04-06,01:00:00,None,None
4,5,Chinese Grand Prix,Shanghai International Circuit,"Shanghai, China",2024-04-21,02:00:00,2024-04-19,22:30:00,None,None,None,None,2024-04-20,02:00:00,2024-04-20,22:00:00
5,6,Miami Grand Prix,Miami International Autodrome,"Miami, USA",2024-05-05,15:00:00,2024-05-03,11:30:00,None,None,None,None,2024-05-04,15:00:00,2024-05-04,11:00:00
6,7,Emilia Romagna Grand Prix,Autodromo Enzo e Dino Ferrari,"Imola, Italy",2024-05-19,08:00:00,2024-05-17,06:30:00,2024-05-17,10:00:00,2024-05-18,05:30:00,2024-05-18,09:00:00,None,None
7,8,Monaco Grand Prix,Circuit de Monaco,"Monte-Carlo, Monaco",2024-05-26,08:00:00,2024-05-24,06:30:00,2024-05-24,10:00:00,2024-05-25,05:30:00,2024-05-25,09:00:00,None,None
8,9,Canadian Grand Prix,Circuit Gilles Villeneuve,"Montreal, Canada",2024-06-09,13:00:00,2024-06-07,12:30:00,2024-06-07,16:00:00,2024-06-08,11:30:00,2024-06-08,15:00:00,None,None
9,10,Spanish Grand Prix,Circuit de Barcelona-Catalunya,"Montmeló, Spain",2024-06-23,08:00:00,2024-06-21,06:30:00,2024-06-21,10:00:00,2024-06-22,05:30:00,2024-06-22,09:00:00,None,None


In [21]:
# PostgreSQL connection details
user = "f1_user"
password = "drivetosurvive"
host = "localhost"
port = "5433"
db = "drive_to_survive"

In [22]:
# Define SQL to drop and recreate the table
create_table_sql = """
DROP TABLE IF EXISTS schedule_2024;
CREATE TABLE schedule_2024 (
    round INTEGER,
    race VARCHAR(255),
    circuit VARCHAR(255),
    location VARCHAR(255),
    race_date DATE,
    race_time TIME,
    first_practice_date DATE,
    first_practice_time TIME,
    second_practice_date DATE,
    second_practice_time TIME,
    third_practice_date DATE,
    third_practice_time TIME,
    qualifying_date DATE,
    qualifying_time TIME,
    sprint_date DATE,
    sprint_time TIME
);
"""

In [23]:
# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

In [24]:
# Execute the table creation SQL
with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()

In [25]:
dtype_mapping = {
    "round": Integer,
    "race": String(255),
    "circuit": String(255),
    "location": String(255),
    "race_date": Date,
    "race_time": Time,
    "first_practice_date": Date,
    "first_practice_time": Time,
    "second_practice_date": Date,
    "second_practice_time": Time,
    "third_practice_date": Date,
    "third_practice_time": Time,
    "qualifying_date": Date,
    "qualifying_time": Time,
    "sprint_date": Date,
    "sprint_time": Time
}

df.to_sql('schedule_2024', engine, if_exists='replace', index=False, dtype=dtype_mapping)

24